In [ ]:
!pip install --quiet asyncpraw
!pip install --quiet nest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 3.3 MB/s eta 0:00:00


In [ ]:
import asyncpraw
import asyncio
import aiohttp
import nest_asyncio

nest_asyncio.apply()

async def get_most_downvoted_comments(subreddit_name, limit=20):
    reddit = asyncpraw.Reddit(
        client_id='PFmQMsDo_N9kdQAshcN8VQ',
        client_secret='48jfkdAA7HghaLCoizMB4jZRPStlHg',
        user_agent='querying_reddit_jae/0.1 by Impressive_Row_3516'
    )

    try:
        subreddit = await reddit.subreddit(subreddit_name)
        comments = []

        async for submission in subreddit.hot(limit=limit):
            try:
                submission = await reddit.submission(id=submission.id)
                submission.comment_sort = 'controversial'
                await submission.comments.replace_more(limit=None)

                def get_all_comments(comment_forest):
                    all_comments = []
                    for comment in comment_forest:
                        all_comments.append(comment)
                        all_comments.extend(get_all_comments(comment.replies))
                    return all_comments

                submission_comments = get_all_comments(submission.comments)
                comments.extend(submission_comments)

            except Exception as e:
                print(f"Error fetching comments for submission {submission.id}: {e}")
                continue

        downvoted_comments = [c for c in comments if hasattr(c, 'score') and c.score < 0]
        downvoted_comments.sort(key=lambda x: x.score)

        for comment in downvoted_comments[:10]:
            try:
                author = comment.author.name if comment.author else "[deleted]"
                print(f"Comment ID: {comment.id}")
                print(f"Author: {author}")
                print(f"Score: {comment.score}")
                print(f"Link: https://reddit.com{comment.permalink}")
                print(f"Comment Body: {comment.body[:300]}...")
                print("------")
            except Exception as e:
                print(f"Error displaying comment {comment.id}: {e}")

    finally:
        await reddit.close()

async def main():
    async with aiohttp.ClientSession() as session:
        await get_most_downvoted_comments('UCSD', limit=50)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

<ipython-input-5-9d77ffdb5cc2>:24: UserWarning: The comments for this submission have already been fetched, so the updated comment_sort will not have any effect.
  submission.comment_sort = 'controversial'


Comment ID: lv4gwb7
Author: spheresickle
Score: -7
Link: https://reddit.com/r/UCSD/comments/1gibx51/upcoming_election/lv4gwb7/
Comment Body: because no matter who wins the US will keep being the bad guys...
------
Comment ID: lv7vxvl
Author: UCSDSuckmyBallz
Score: -4
Link: https://reddit.com/r/UCSD/comments/1git3iy/esports_program/lv7vxvl/
Comment Body: Tec center is so greasy. Only degens play at it....
------
Comment ID: lv2i2us
Author: XarkXD
Score: -4
Link: https://reddit.com/r/UCSD/comments/1gi3ccs/hi/lv2i2us/
Comment Body: I can't believe you said that! I'm reporting you...
------
Comment ID: lv82k95
Author: RandomsHater567
Score: -3
Link: https://reddit.com/r/UCSD/comments/1gibx51/upcoming_election/lv82k95/
Comment Body: Most judges are extreme left wing its aight dude u gonna make out the hood...
------
Comment ID: lv5j0bj
Author: Little-School2823
Score: -1
Link: https://reddit.com/r/UCSD/comments/1gibx51/upcoming_election/lv5j0bj/
Comment Body: please give me an example of a 